#### Student Name: Mai Ngo
#### Course Name and Number: DSC 575 Intelligent Information Retrieval - SEC 801
#### Assignment 2 - Inverted index
#### Date: 1/28/2024

In [1]:
import nltk
import csv
import math
import pandas as pd
import numpy as np
import warnings
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter

## 1. Position Indices

### Read file.
#### Store positional index as a text file. 
#### Store as a nested dictionary. With postings as values (dictionary type) and terms as keys.

In [2]:
positionalIndex = {}

with open ('Position Indices.txt') as infile:
    content = infile.read()
    lines = content.split(';\n')

    for line in lines:
        parts = line.split(':')
        term = parts[0].strip()
        #The rest of each term positional index.
        postings = parts[1:]
        
        #First: extract positional index elements to a list.   
        pos_inxList=[]
        #Append the first element aka. doc2
        pos_inxList.append(postings[0].strip(' '))
        
        for element in postings: 
            element = element.strip(' ').split(' ')
            #Because elements are not splitted correctly.
            if len(element) == 2:
                babyPostings = element[0].strip(';')
                pos_inxList.append(babyPostings)
                docNum = element[1]
                pos_inxList.append(docNum)
        pos_inxList.append(element[-1])
        
        #Second: re-arrange positional index elements to a dictionary.
        #Keys: Document numbers | Values: positional index.
        pos_inxDict={}
        for i in range(0, len(pos_inxList), 2):
            docNumber = pos_inxList[i]
            rawPositions = pos_inxList[i + 1].strip('<>;')
            positions = list(map(int, rawPositions.split(',')))
            pos_inxDict[docNumber] = positions
        
        #Add to the master dictionary.
        positionalIndex[term] = pos_inxDict

print(f'Positional Index data:')
print(positionalIndex)

Positional Index data:
{'angels': {'2': [36, 174, 252, 651], '4': [12, 22, 102, 432], '7': [17]}, 'fools': {'2': [1, 17, 74, 222], '4': [8, 78, 108, 458], '7': [3, 13, 23, 193]}, 'fear': {'2': [87, 704, 722, 901], '4': [13, 43, 113, 433], '7': [18, 328, 528]}, 'in': {'2': [3, 37, 76, 444, 851], '4': [10, 20, 110, 470, 500], '7': [5, 15, 25, 195]}, 'rush': {'2': [2, 66, 194, 321, 702], '4': [9, 69, 149, 429, 569], '7': [4, 14, 404]}, 'to': {'2': [47, 86, 234, 999], '4': [14, 24, 774, 944], '7': [199, 319, 599, 709]}, 'tread': {'2': [57, 94, 333], '4': [15, 35, 155], '7': [20, 320]}, 'where': {'2': [67, 124, 393, 1001], '4': [11, 41, 101, 421, 431], '7': [15, 35, 735]}}


### a. Which document(s) (if any) match each of the following queries where each expression within quotes is a phrase query?  At which positions do the queries match?

 #### i. “fools rush in”

In [3]:
query1 = 'fools rush in'
queryTerms1 = query1.split()

matchingDocs = []

#Get neccessary items only (based on query terms) from positionalIndex dictionary.
for docNum, positions in positionalIndex[queryTerms1[0]].items():
    
    #Get positional index of each term in the query.
    pos1stTerm1 = positionalIndex[queryTerms1[0]].get(docNum, [])
    pos2ndTerm1 = positionalIndex[queryTerms1[1]].get(docNum, [])
    pos3rdTerm1 = positionalIndex[queryTerms1[2]].get(docNum, [])

    #Basic logic positional index of 1st term + 1 = positional index of 2nd term.
    for posFirst in pos1stTerm1:
        if posFirst + 1 in pos2ndTerm1 and posFirst + 2 in pos3rdTerm1:
            matchingDocs.append((docNum, posFirst))
            
for pairs in matchingDocs:
    print(f'Matching found in document {pairs[0]} at position index {pairs[1]}.')

Matching found in document 2 at position index 1.
Matching found in document 4 at position index 8.
Matching found in document 7 at position index 3.
Matching found in document 7 at position index 13.


ii. “fools rush in” AND “angels fear to tread”.

In [4]:
#Same logic as previous part.
query2 = 'angels fear to tread'
queryTerms2 = query2.split()

matchingDocs2 = []

for docNum, positions in positionalIndex[queryTerms2[0]].items():
  
    pos1stTerm2 = positionalIndex[queryTerms2[0]].get(docNum, [])
    pos2ndTerm2 = positionalIndex[queryTerms2[1]].get(docNum, [])
    pos3rdTerm2 = positionalIndex[queryTerms2[2]].get(docNum, [])
    pos4thTerm2 = positionalIndex[queryTerms2[3]].get(docNum, [])

    for posFirst in pos1stTerm2:
        if posFirst + 1 in pos2ndTerm2 and posFirst + 2 in pos3rdTerm2 and posFirst + 3 in pos4thTerm2:
            matchingDocs2.append((docNum, posFirst))

for pairs in matchingDocs:
    for pairs2 in matchingDocs2:
        #Same document.
        if pairs[0] == pairs2[0]:
            print(f'Matching found in document {pairs[0]} at position indices {pairs[1]} and {pairs2[1]}.')

Matching found in document 4 at position indices 8 and 12.


### b. There is something wrong with this positional index. What is the problem?
#### One position has two words registered --> IMPOSSIBLE.

In [5]:
def checkPosition(positionalIndex):
    '''Check if there are multiple words register to one position within a documentt.'''
    
    #Re-arrange the original positional index dictionary.
    #Still as a nested dictionary.
    posDict = {}

    for term, posIndex in positionalIndex.items():
        for document, index in posIndex.items():
            for idx in index:
                #New index position, add in.
                if idx not in posDict:
                    #Positions as keys | terms & doc number as values (dictionary type).
                    posDict[idx] = {term: document}
                
                #Index position already exist in the dictionary.
                else:
                    #Check if current document exists as well.
                    if document in posDict[idx].values():
                        
                        #Retrieve the current item with same index & document.
                        for items in posDict[idx].items():
                            if items[1] == document:
                                currentTerm = items[0]
                        print(f"Document {document}: Duplicate position {idx} for terms '{term}' and '{currentTerm}'.")
                    
                    #If different document, add to the dictionary.
                    else:
                        posDict[idx][term] = document
checkPosition(positionalIndex)

Document 7: Duplicate position 15 for terms 'where' and 'in'.


## 2. TFiDF Weighting

### a. Compute the new weights for all the terms in documents DOC3 and DOC7 using the tf x idf approach.  Use the raw term frequency for TF (for both documents and query) and the log of base 10 of N/df for IDF.  

#### Create document-term matrix. Documents as rows, terms as columns. 

In [6]:
#Read: 3 times Term2 occurs in DOC1.
doc_termMatrix = np.array([[0, 3, 1, 0, 0, 2, 1, 0], 
                           [5, 0, 0, 0, 3, 0, 0, 2], 
                           [3, 0, 4, 3, 4, 0, 0, 5], 
                           [1, 8, 0, 3, 0, 1, 4, 0],
                           [0, 1, 0, 0, 0, 5, 4, 2],
                           [2, 0, 2, 0, 0, 4, 0, 1], 
                           [2, 5, 0, 3, 0, 1, 4, 2],
                           [3, 3, 0, 2, 0, 0, 1, 3],
                           [0, 0, 3, 3, 3, 0, 0, 0],
                           [1, 0, 5, 0, 2, 4, 0, 2]])
print(f'Document-term Matrix shape: {doc_termMatrix.shape}')
print(f'Number of terms: {doc_termMatrix.shape[1]}')
print(f'Number of documents: {doc_termMatrix.shape[0]}')

Document-term Matrix shape: (10, 8)
Number of terms: 8
Number of documents: 10


#### Get document frequency

In [7]:
#Transpose the data to term-document matrix.
#Terms are row, documents are columns. 
term_docMatrix = doc_termMatrix.T

#Get document frequency/count of each term. 
#Counting non-zero elements for each row - axis = 1.
#Reshaped into one column vector.
#Read: Term 1 occurs in 7 out of 10 documents.
docNum_perTerm = np.count_nonzero(term_docMatrix, axis=1).reshape(-1, 1)
print('Number of documents a term exists in:')
print(docNum_perTerm)

Number of documents a term exists in:
[[7]
 [5]
 [5]
 [5]
 [4]
 [6]
 [5]
 [7]]


#### Create base matrix with each element is total number of documents N.

In [8]:
#Get total document counts.
documentNum = term_docMatrix.shape[1]
documentNum_Matrix=np.ones(np.shape(term_docMatrix), dtype=float)*documentNum
print(documentNum_Matrix)

[[10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]]


#### Calculate IDF following the instruction. Using log base of 10.

In [9]:
#Set print out option, 5 decimals.
np.set_printoptions(precision=5,suppress=True,linewidth=120)

#ID formula log10(N/df)
#documentNum_Matrix: total number of document. 
#docNum_perTerm: number of document a term exists in.
IDF = np.log10(np.divide(documentNum_Matrix, docNum_perTerm))
print('IDF:')
print(IDF)

IDF:
[[0.1549  0.1549  0.1549  0.1549  0.1549  0.1549  0.1549  0.1549  0.1549  0.1549 ]
 [0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103]
 [0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103]
 [0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103]
 [0.39794 0.39794 0.39794 0.39794 0.39794 0.39794 0.39794 0.39794 0.39794 0.39794]
 [0.22185 0.22185 0.22185 0.22185 0.22185 0.22185 0.22185 0.22185 0.22185 0.22185]
 [0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103 0.30103]
 [0.1549  0.1549  0.1549  0.1549  0.1549  0.1549  0.1549  0.1549  0.1549  0.1549 ]]


#### Calculate TFxIDF with TF as raw term frequency.

In [10]:
#Compute the TFxIDF values for document-term entry.
TFIDF = term_docMatrix * IDF
print(TFIDF)

[[0.      0.77451 0.46471 0.1549  0.      0.3098  0.3098  0.46471 0.      0.1549 ]
 [0.90309 0.      0.      2.40824 0.30103 0.      1.50515 0.90309 0.      0.     ]
 [0.30103 0.      1.20412 0.      0.      0.60206 0.      0.      0.90309 1.50515]
 [0.      0.      0.90309 0.90309 0.      0.      0.90309 0.60206 0.90309 0.     ]
 [0.      1.19382 1.59176 0.      0.      0.      0.      0.      1.19382 0.79588]
 [0.4437  0.      0.      0.22185 1.10924 0.88739 0.22185 0.      0.      0.88739]
 [0.30103 0.      0.      1.20412 1.20412 0.      1.20412 0.30103 0.      0.     ]
 [0.      0.3098  0.77451 0.      0.3098  0.1549  0.3098  0.46471 0.      0.3098 ]]


#### Convert back to document-term matrix.

In [11]:
doc_termTFIDF = TFIDF.transpose()
doc_termTFIDF

array([[0.     , 0.90309, 0.30103, 0.     , 0.     , 0.4437 , 0.30103, 0.     ],
       [0.77451, 0.     , 0.     , 0.     , 1.19382, 0.     , 0.     , 0.3098 ],
       [0.46471, 0.     , 1.20412, 0.90309, 1.59176, 0.     , 0.     , 0.77451],
       [0.1549 , 2.40824, 0.     , 0.90309, 0.     , 0.22185, 1.20412, 0.     ],
       [0.     , 0.30103, 0.     , 0.     , 0.     , 1.10924, 1.20412, 0.3098 ],
       [0.3098 , 0.     , 0.60206, 0.     , 0.     , 0.88739, 0.     , 0.1549 ],
       [0.3098 , 1.50515, 0.     , 0.90309, 0.     , 0.22185, 1.20412, 0.3098 ],
       [0.46471, 0.90309, 0.     , 0.60206, 0.     , 0.     , 0.30103, 0.46471],
       [0.     , 0.     , 0.90309, 0.90309, 1.19382, 0.     , 0.     , 0.     ],
       [0.1549 , 0.     , 1.50515, 0.     , 0.79588, 0.88739, 0.     , 0.3098 ]])

#### Output as pandas dataframe. 

In [12]:
columnNames = ['Term1', 'Term2', 'Term3', 'Term4', 'Term5', 'Term6', 'Term7', 'Term8']
indexNames = [f'Doc{i+1}' for i in range(len(doc_termTFIDF))]
doc_termTFIDF_pd = pd.DataFrame(doc_termTFIDF, columns=columnNames, index=indexNames)
print('TF-IDF weights:')
doc_termTFIDF_pd

TF-IDF weights:


,Term1,Term2,Term3,Term4,Term5,Term6,Term7,Term8
Doc1,0.000000,0.90309,0.30103,0.00000,0.00000,0.443697,0.30103,0.000000
Doc2,0.774510,0.00000,0.00000,0.00000,1.19382,0.000000,0.00000,0.309804
Doc3,0.464706,0.00000,1.20412,0.90309,1.59176,0.000000,0.00000,0.774510
Doc4,0.154902,2.40824,0.00000,0.90309,0.00000,0.221849,1.20412,0.000000
Doc5,0.000000,0.30103,0.00000,0.00000,0.00000,1.109244,1.20412,0.309804
Doc6,0.309804,0.00000,0.60206,0.00000,0.00000,0.887395,0.00000,0.154902
Doc7,0.309804,1.50515,0.00000,0.90309,0.00000,0.221849,1.20412,0.309804
Doc8,0.464706,0.90309,0.00000,0.60206,0.00000,0.000000,0.30103,0.464706
Doc9,0.000000,0.00000,0.90309,0.90309,1.19382,0.000000,0.00000,0.000000
Doc10,0.154902,0.00000,1.50515,0.00000,0.79588,0.887395,0.00000,0.309804


### New weights for all the terms in documents DOC3.

In [13]:
doc3 = doc_termTFIDF_pd.loc['Doc3']
print('New weights for all the terms in documents DOC3:')
doc3

New weights for all the terms in documents DOC3:


Term1    0.464706
Term2    0.000000
Term3    1.204120
Term4    0.903090
Term5    1.591760
Term6    0.000000
Term7    0.000000
Term8    0.774510
Name: Doc3, dtype: float64

### New weights for all the terms in documents DOC7.

In [14]:
doc7 = doc_termTFIDF_pd.loc['Doc7']
print('New weights for all the terms in documents DOC7:')
doc7

New weights for all the terms in documents DOC7:


Term1    0.309804
Term2    1.505150
Term3    0.000000
Term4    0.903090
Term5    0.000000
Term6    0.221849
Term7    1.204120
Term8    0.309804
Name: Doc7, dtype: float64

### b. Using cosine as the measure, which document is the most relevant to DOC7?  Show your calculations.  

In [15]:
def cosineSimilarity (instance1, instance2):
    '''Return Cosine similarity distance between two instance.'''
    
    #Find the vector norm for each instance. 
    instance1_norm = np.linalg.norm(instance1)
    instance2_norm = np.linalg.norm(instance2)
          
    #Compute Cosine: divide the dot product of the predicted instance versus each document instance by the product of the two norms.
    cosine = np.dot(instance1, instance2)/(instance1_norm * instance2_norm)
    
    return cosine    

In [16]:
def cosine_simDoc(doc, weightMatrix):
    '''Get the most similar document based on Cosine similarity.'''
    '''Using raw term frequnecy and pandas data frame.'''
    
    cosSim_res = []
    
    for index, row in weightMatrix.iterrows():
        cosineSim = cosineSimilarity(doc, row.values)
        cosSim_res.append((index, cosineSim))
    
    #Sort the similarities in descending order.
    cosSim_res.sort(key=lambda sim: sim[1], reverse=True)
    
    #Return 2nd similar item. First item is the document itself.
    return cosSim_res[1]

In [17]:
doc_termMatrix_pd = pd.DataFrame(doc_termMatrix)
doc7Raw = doc_termMatrix_pd.iloc[6]
cosineSimilar = cosine_simDoc(doc7Raw, doc_termMatrix_pd)
print(f"The most relevant document to Doc7 is Doc{cosineSimilar[0] + 1} with a Cosine similarity of {cosineSimilar[1]:.4f}")

The most relevant document to Doc7 is Doc4 with a Cosine similarity of 0.9280


### 3. Inverted Index

### Import dataset as pandas dataframe.

In [18]:
ted = pd.read_csv('ted_main.csv', encoding='utf-8')
print(f'Numnber of rows and columns of the ted data: {ted.shape}')
#Read data as Pandas dataframe and get number of rows and columns. 

Numnber of rows and columns of the ted data: (2550, 17)


In [19]:
ted.head(3)
#Get the first 3 rows. 

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292


#### Extract 'description' 2nd column.

In [20]:
tedDesc = ted[['description']]
print(f'Number of row of the description column: {tedDesc.shape}')

Number of row of the description column: (2550, 1)


In [21]:
tedDesc.iloc[1]

description    With the same humor and humanity he exuded in ...
Name: 1, dtype: object

#### Extract 'url' 16th column.

In [22]:
tedURL = ted[['url']]
print(f'Number of row of the url column: {tedURL.shape}')

Number of row of the url column: (2550, 1)


In [23]:
tedURL.iloc[1]

url    https://www.ted.com/talks/al_gore_on_averting_...
Name: 1, dtype: object

#### Pre-process following scheme [C]: Word tokenization + Case folding (lower-case) + Stopword filtering + Non-alphabet filtering + Porter stemming.

In [24]:
#Set to store English stop word for efficiency.
stopwordsSet = set(stopwords.words('english'))

#Porter stemmer object.
porterStemmer = nltk.PorterStemmer()

def processDescription(description):
    '''Apply Word tokenization + Case folding (lower-case) + Stopword filtering + Non-alphabet filtering + Porter stemming to a string.'''
    
    tokens = [porterStemmer.stem(token) for token in word_tokenize(description.lower())
              if token.isalpha() and token not in stopwordsSet]
    
    #Using Counter for dictionary output.
    return Counter(tokens)

### Data frame needed for this problem.

In [25]:
tedShort = ted[['url','description']]
tedShort.head(3)

,url,description
0,https://www.ted.com/talks/ken_robinson_says_sc...,Sir Ken Robinson makes an entertaining and pro...
1,https://www.ted.com/talks/al_gore_on_averting_...,With the same humor and humanity he exuded in ...
2,https://www.ted.com/talks/david_pogue_says_sim...,New York Times columnist David Pogue takes aim...


In [26]:
warnings.filterwarnings("ignore")
#Apply the text processing function to each description row.
tedShort.loc[:, 'processedDesc'] = tedShort['description'].apply(processDescription)

In [27]:
tedShort.head(2)

,url,description,processedDesc
0,https://www.ted.com/talks/ken_robinson_says_sc...,Sir Ken Robinson makes an entertaining and pro...,"{'sir': 1, 'ken': 1, 'robinson': 1, 'make': 1,..."
1,https://www.ted.com/talks/al_gore_on_averting_...,With the same humor and humanity he exuded in ...,"{'humor': 1, 'human': 1, 'exud': 1, 'inconveni..."


#### Cross tabulation, each word in processed Description as a row with corresponding URL.

In [28]:
#Look at processed desciption per row, then iterate through each word.
#Assign to tuple (A SINGLE WORD, corresponding word count within URL document, corresponding URL).

pdWord = pd.DataFrame([(word, word_count, tedShort.loc[index, 'url']) for index, row in tedShort.iterrows() 
                       for word, word_count in row['processedDesc'].items()], columns=['word', 'word_count', 'url'])
pdWord.head(3)

,word,word_count,url
0,sir,1,https://www.ted.com/talks/ken_robinson_says_sc...
1,ken,1,https://www.ted.com/talks/ken_robinson_says_sc...
2,robinson,1,https://www.ted.com/talks/ken_robinson_says_sc...


### 1. A file that maps each term name to: the term ID and the document frequency (i.e., the number of files the term occurred in). Assign a series of positive integers (1-based) to the term names. Store each term name per line. Name the file "TED_term_index.csv". The first couple of lines in the file should look like this:

#### Group by 'word' and count how many URL associated with a word.

In [29]:
#size() - count occurrences grouped by one word.
pdWordcounts = pdWord.groupby('word').size().reset_index(name='count')
pdWordcounts['term_ID'] = pdWordcounts.index + 1
pdWordcounts.head(3)

,word,count,term_ID
0,a,2,1
1,aakash,1,2
2,aala,1,3


#### Write to a CSV file.

In [30]:
with open('TED_term_index.csv', 'w', newline='', encoding='utf-8') as outFile:
    outFile.write("word, term_ID, count\n")  
    for row in pdWordcounts.itertuples(index=False):
        template = f"{row.word}, {row.term_ID}, {row.count}\n"
        outFile.write(template)

print("Successfully written!")

Successfully written!


### 2. A file that maps each document ID to: the document name. Assign a series of positive integers (1-based) to the documents.Store each document ID per line. Name the file "TED_doc_index.csv". The first couple of lines in the file should look like this:

#### Extract url column and assign doc_ID.

In [31]:
#Add document index column.
tedURL['doc_ID'] = tedURL.index + 1
tedURL.head(3)

,url,doc_ID
0,https://www.ted.com/talks/ken_robinson_says_sc...,1
1,https://www.ted.com/talks/al_gore_on_averting_...,2
2,https://www.ted.com/talks/david_pogue_says_sim...,3


#### Write to a CSV file.

In [32]:
with open('TED_doc_index.csv', 'w', newline='', encoding='utf-8') as outFile:
    outFile.write("doc_ID, url\n")  
    for row in tedURL.itertuples(index=False):
        template = f"{row.doc_ID}, {row.url}"
        outFile.write(template)

print("Successfully written!")

Successfully written!


### 3. The inverted index file that maps each term ID to its postings list. Each posting should contain a document ID and the term frequency in that document. Store one term ID per line. Name the file "TED_inverted_index.csv". The first couple of lines in the file should look like this:

#### Merge term_ID, doc_ID to main word count pandas data frame. 

In [33]:
#Create a copy for this problem.
pdWord_inverted=pdWord

#Merge each word to correpoding term_ID.
pdWord_inverted = pdWord_inverted.merge(pdWordcounts[['word', 'term_ID']], on='word', how='left')

#Merge each doc to correpoding doc_ID.
pdWord_inverted = pdWord_inverted.merge(tedURL[['url', 'doc_ID']], on='url', how='left')

#Sorted in ascending order. 
pdWord_inverted = pdWord_inverted.sort_values(by=['term_ID', 'doc_ID'])

#Display the updated pdWord DataFrame
pdWord_inverted.head(3)

,word,word_count,url,term_ID,doc_ID
25373,a,1,https://www.ted.com/talks/lucien_engelen_crowd...,1,1146
65127,a,1,https://www.ted.com/talks/todd_scott_an_interg...,1,2429
45972,aakash,1,https://www.ted.com/talks/aakash_odedra_a_danc...,2,1878


#### Write to a CSV file with print template. 

In [34]:
invertedIndex_lines = []

#Create template line. 
for termID, postings in pdWord_inverted.groupby('term_ID'):
    template = f"{termID}, " + ', '.join(f"{docID}, {wordCount}" for (docID, wordCount) in postings[['doc_ID', 'word_count']].itertuples(index=False))
    invertedIndex_lines.append(template)
    
with open('TED_inverted_index.csv', 'w', newline='', encoding='utf-8') as outFile:
    outFile.write('\n'.join(invertedIndex_lines))

print("Successfully written!")

Successfully written!


## Boolean conjunctive queries.
#### Work on pdWord_inverted pandas data frame.

In [35]:
def booleanAND(term1, term2, dataFrame):
    '''Apply boolean query search using AND gate for 2 terms'''
    '''Return number of documents satisified condition and breakdown data.'''
    
    #Porter stemmer object.
    porterStemmer = nltk.PorterStemmer()

    stemmedTerm1 = porterStemmer.stem(term1)
    stemmedTerm2 = porterStemmer.stem(term2)
  
    #Extract only rows with query term.
    res1 = dataFrame[dataFrame['word'].isin([stemmedTerm1, stemmedTerm2])]
    #Filter only rows with doc_ID occurs more than twice for 1 word.
    res2 = res1.groupby('doc_ID').filter(lambda x: len(x) >= 2)
    
    #Compile output data frame.
    res3 = pd.DataFrame(columns=['doc_ID', 'url', 'term_frequency'])
    for doc_ID, postings in res2.groupby('doc_ID'):
        #Make sure same URL has 1 doc_ID.
        url = postings['url'].iloc[0]  
        tf = postings[['word', 'word_count']].values.tolist()
        res3 = res3.append({'doc_ID': doc_ID, 'url': url, 'term_frequency': tf}, ignore_index=True)
    
    docNum = res3.shape[0]
    
    print(f"Number of documents have both terms '{term1}' AND '{term2}' are {docNum}.")
    
    #Print out res2 for code verification. Technically we do not need. 
    return res2, res3

### a. 'climate' AND 'change'

In [36]:
climateChange2, climateChange3 = booleanAND('climate', 'change', pdWord_inverted)

Number of documents have both terms 'climate' AND 'change' are 35.


In [37]:
print("Break down chart for query 'climate' AND 'change'.")
climateChange3

Break down chart for query 'climate' AND 'change'.


,doc_ID,url,term_frequency
0,2,https://www.ted.com/talks/al_gore_on_averting_...,"[[chang, 1], [climat, 1]]"
1,110,https://www.ted.com/talks/john_doerr_sees_salv...,"[[chang, 1], [climat, 1]]"
2,161,https://www.ted.com/talks/david_keith_s_surpri...,"[[chang, 1], [climat, 1]]"
3,215,https://www.ted.com/talks/al_gore_s_new_thinki...,"[[chang, 1], [climat, 1]]"
4,492,https://www.ted.com/talks/gordon_brown\n,"[[chang, 1], [climat, 1]]"
5,505,https://www.ted.com/talks/cary_fowler_one_seed...,"[[chang, 1], [climat, 1]]"
6,510,https://www.ted.com/talks/james_balog_time_lap...,"[[chang, 1], [climat, 1]]"
7,551,https://www.ted.com/talks/rachel_pike_the_scie...,"[[chang, 1], [climat, 1]]"
8,742,https://www.ted.com/talks/lewis_pugh_s_mind_sh...,"[[chang, 1], [climat, 1]]"
9,751,https://www.ted.com/talks/lee_hotz_inside_an_a...,"[[chang, 1], [climat, 1]]"


### b.'climate' AND 'fuel'

In [38]:
climateFuel2, climateFuel3 = booleanAND('climate', 'fuel', pdWord_inverted)

Number of documents have both terms 'climate' AND 'fuel' are 1.


In [39]:
print("Break down chart for query 'climate' AND 'fuel'.")
climateFuel3

Break down chart for query 'climate' AND 'fuel'.


,doc_ID,url,term_frequency
0,2260,https://www.ted.com/talks/monica_araya_a_small...,"[[climat, 1], [fuel, 1]]"


### c. 'artificial' AND 'intelligence'

In [40]:
artiIntel2, artiIntel3 = booleanAND('artificial', 'intelligence', pdWord_inverted)

Number of documents have both terms 'artificial' AND 'intelligence' are 5.


In [41]:
print("Break down chart for query 'artificial' AND 'intelligence'.")
artiIntel3

Break down chart for query 'artificial' AND 'intelligence'.


,doc_ID,url,term_frequency
0,1268,https://www.ted.com/talks/peter_norvig_the_100...,"[[artifici, 1], [intellig, 1]]"
1,1966,https://www.ted.com/talks/nick_bostrom_what_ha...,"[[artifici, 1], [intellig, 2]]"
2,2319,https://www.ted.com/talks/tim_leberecht_4_ways...,"[[artifici, 1], [intellig, 1]]"
3,2387,https://www.ted.com/talks/grady_booch_don_t_fe...,"[[artifici, 1], [intellig, 1]]"
4,2548,https://www.ted.com/talks/radhika_nagpal_what_...,"[[artifici, 1], [intellig, 2]]"


### d. 'giant' AND 'troll'

In [42]:
giantTroll2, giantTroll3 = booleanAND('giant', 'troll', pdWord_inverted)

Number of documents have both terms 'giant' AND 'troll' are 0.


In [43]:
print("Break down chart for query 'giant' AND 'troll'.")
giantTroll3

Break down chart for query 'giant' AND 'troll'.


,doc_ID,url,term_frequency


### Code verification.
#### Let's use 'climate' AND 'change' with most output.  

In [44]:
#climateChange2 should give all doc_ID that both query words existed in. 
climateChange2.head(4)

,word,word_count,url,term_ID,doc_ID
27,chang,1,https://www.ted.com/talks/al_gore_on_averting_...,1561,2
2032,chang,1,https://www.ted.com/talks/john_doerr_sees_salv...,1561,110
2946,chang,1,https://www.ted.com/talks/david_keith_s_surpri...,1561,161
3964,chang,1,https://www.ted.com/talks/al_gore_s_new_thinki...,1561,215


In [45]:
#Sort by doc_ID, expectation would be every two rows same doc_ID. 
validate = climateChange2.sort_values(by='doc_ID')
#At this point I can confirm my code and logic are correct. 
validate.head(10)

,word,word_count,url,term_ID,doc_ID
27,chang,1,https://www.ted.com/talks/al_gore_on_averting_...,1561,2
26,climat,1,https://www.ted.com/talks/al_gore_on_averting_...,1739,2
2032,chang,1,https://www.ted.com/talks/john_doerr_sees_salv...,1561,110
2031,climat,1,https://www.ted.com/talks/john_doerr_sees_salv...,1739,110
2946,chang,1,https://www.ted.com/talks/david_keith_s_surpri...,1561,161
2945,climat,1,https://www.ted.com/talks/david_keith_s_surpri...,1739,161
3964,chang,1,https://www.ted.com/talks/al_gore_s_new_thinki...,1561,215
3963,climat,1,https://www.ted.com/talks/al_gore_s_new_thinki...,1739,215
9687,climat,1,https://www.ted.com/talks/gordon_brown\n,1739,492
9688,chang,1,https://www.ted.com/talks/gordon_brown\n,1561,492
